# Compute a lower bound for the new TAP-C

To compute a lower bound, we first find a dual solution, then use it to derive a lower bound as sensitivity analysis.

In [1]:
import numpy as np

In [2]:
import cvxpy as cvx

In [3]:
import Frank_Wolf_solver
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network_name = Brae

eps=1e-8
nb_iter = 1000
graph, demand = Frank_Wolf_solver.load_network(network_name)
c = -1
if network_name == Brae:
    demand[0][2] = 10
    c = [11, 11, 2, 11, 11]
path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver.Frank_Wolf_solver(graph, demand, eps, nb_iter, c) # you can add a capacity
print(path_flow_matrix)
print(tt_f)
print(delta)
print(delta @ tt_f)

[1.99999806 4.19310699 3.80689495]
[1.6193105 2.        0.25      2.        1.5806893]
  (0, 0)	1.0
  (0, 2)	1.0
  (0, 4)	1.0
  (1, 0)	1.0
  (1, 3)	1.0
  (2, 1)	1.0
  (2, 4)	1.0
[3.44999981 3.6193105  3.5806893 ]


In [19]:
def lower_bound(f, h, t, Delta, route2od, u_old, u_new, nk):
    """ 
    function that compute a lower bound with updated capacity given the original solution of a TAP-C
    input:
    f(vector): primal solution of link flow
    h(vector): primal solution of route flow
    t(ventor): link travel time of f
    Delta(matrix)[r,a]: 1 if route r use link a; 0 otherwise. 
    route2od(vector): mapping the route index to the index of o-d pairs
    u_old(vector): original capacity of links
    u_new(vector): new capacity of links   
    """
    nr, na = Delta.shape # number of non-zero routes and number of links 
    c = Delta.dot(t)
    
    lambda_link = cvx.Variable(na)
    pi = cvx.Variable(nk)
    
    obj = cvx.Maximize((u_old - u_new) * lambda_link)
    
    
    float_appro = 1e-3
    
    constr_1 = [lambda_link[i]==0 for i in range(na) if f[i] < u_old[i] - float_appro]
    constr_2 = [lambda_link[i]>=0 for i in range(na)]
    constr_3 = [c[r]+Delta[r]*lambda_link == pi[route2od[r]] for r in range(nr) if h[r]>0]
    constr_4 = [c[r]+Delta[r]*lambda_link >= pi[route2od[r]] for r in range(nr)]
    
    constr = constr_1 + constr_2 + constr_3 + constr_4
    
    problem = cvx.Problem(obj, constr)
    bound = problem.solve()
    
    return bound, lambda_link.value, pi.value

In [12]:
lower_bound(f, h, tt, Delta, route2od, u_old, u_new, nk)

[3.4499998059324355, 3.619310504880816, 3.5806893010516196]


IndexError: Too many indices for expression.

In [65]:
f = np.array([1.0,1.1,0.1])
h = np.array([1.0, 0.1])
t = np.array([3.0, 2.1, 9.0])
Delta = np.array([[1,1,0], [0,1,1]])
route2od = [0,0]
u_old = np.array([1.0, 2.0, 3.0])
u_new = np.array([0.5, 0.9, 3.0])
nk = 1

In [20]:
h = path_flow_matrix
f = delta.T @ path_flow_matrix
Delta = delta.toarray()
u_old = np.array(c)
u_new = np.array(c) - np.array([0, 0, 1, 0, 0])
route2od = [0,0,0]
nk = 1
tt = tt_f

lower_bound(f, h, tt, Delta, route2od, u_old, u_new, nk)

(-inf, None, None)

In [7]:
debug = True
# graph and demand gives the graph and the demand. The 
network = 'data/I210'
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

# The following is some work to get a nice way to present the od and the graph
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])
if debug:
    print(demand)

# in the case where the index of the od pairs does not begin by 0, we rename the od pairs
first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
graph[:,1] = graph[:,1]-first_index_od
graph[:,2] = graph[:,2]-first_index_od
demand[:,0] = demand[:,0] - first_index_od
demand[:,1] = demand[:,1] - first_index_od
if debug:
    print(first_index_od)
    print(graph)

[[1.9e+01 2.0e+01 2.5e+04]]
1.0
[[0.0000e+00 0.0000e+00 1.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [1.0000e+00 0.0000e+00 6.0000e+00 1.1500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7600e+00]
 [2.0000e+00 1.0000e+00 2.0000e+00 1.3800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.3120e+00]
 [3.0000e+00 1.0000e+00 7.0000e+00 1.1400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7360e+00]
 [4.0000e+00 2.0000e+00 3.0000e+00 1.6800e+00 0.0000e+00 0.0000e+00
  0.0000e+00 4.0320e+00]
 [5.0000e+00 2.0000e+00 8.0000e+00 1.1300e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.7120e+00]
 [6.0000e+00 3.0000e+00 4.0000e+00 1.2600e+00 0.0000e+00 0.0000e+00
  0.0000e+00 3.0240e+00]
 [7.0000e+00 3.0000e+00 9.0000e+00 1.0500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.5200e+00]
 [8.0000e+00 4.0000e+00 5.0000e+00 2.2500e+00 0.0000e+00 0.0000e+00
  0.0000e+00 5.4000e+00]
 [9.0000e+00 4.0000e+00 1.0000e+01 1.0400e+00 0.0000e+00 0.0000e+00
  0.0000e+00 2.4960e+00]
 [1.0000e+01 5.0000e+00 1.1000e+01 1.0

In [8]:
# Here are real values output from the Frank Wolf solver without capacity contraints.
# I have to work on the capacity constraints now.
f = np.array([0.0, 0.0, 6260.993213452754, 6260.993213452754, 38.89901571580986, 6299.892229168564, 0.0, 6299.892229168564, 0.0, 6299.892229168564, 0.0, 0.0, 0.0, 25000.0, 0.0, 25000.0, 0.0, 0.0, 12205.86061466584, 38.89901571580986, 0.0, 12205.86061466584, 0.0, 0.0, 12205.86061466584, 0.0, 6299.892229168564, 12205.86061466584, 6494.247156165595, 0.0, 6533.146171881404, 0.0, 0.0, 6533.146171881404, 0.0, 6494.247156165595, 0.0, 6494.247156165595, 0.0, 6494.247156165595, 25000.0])
tt = np.array([1.38, 1.15, 5089361032701062.0, 4204254766144356.5, 9231559.039423268, 4271910701258548.0, 1.26, 3969474545417235.0, 2.25, 3931670025937070.5, 1.09, 1.15, 0.48, 3.375e+17, 1.14, 720000000000000.5, 0.39, 1.13, 49434806976970.27, 2088090.735107644, 1.05, 36649943103615.9, 0.48, 1.04, 65628967883219.164, 0.5, 4120692623337891.5, 2.5037025085318704e+16, 1536841197275133.8, 1.38, 1705160836426404.5, 1.38, 0.38, 7345308218452204.0, 0.48, 5336254157205326.0, 0.5, 9477187383196660.0, 0.47, 1502689170669.2395, 435000000000000.3])
h = np.array([12205.86061466584, 6494.247156165595, 6260.993213452754, 38.89901571580986])
delta = np.array([[13, 15, 18, 21, 24, 27, 40],
    [13, 15, 30, 33, 35, 37, 39, 28, 40],
    [13, 15, 2, 3, 5, 7, 9, 26, 40],
    [13, 15, 30, 33, 19, 4, 5, 7, 9, 26, 40]])
u_old = np.array([1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308, 1.7976931348623157e+308])
# for each path, route2od gives the row of the demand vector where you can find the od
# to be more precise, the path delta[i] has for origin: demand[route2od[i]][0], and for destination: demand[route2od[i]][1]
route2od = [0,0,0,0]